# Dataframe Cleaning & Feature Extraction

The <b> purpose </b>of this notebook is to merge and clean dataframes - all of theses steps will assist in feeding keywords into the Twitter API and the Gephi platform. 

## Libraries

In [59]:
import pandas as pd
import numpy as np
from pprint import pprint
import spacy
import json
import os
import matplotlib.pyplot as plt
import matplotlib
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import models
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('wordnet')
import pyLDAvis.gensim
import pickle

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/celinasprague/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Loading Exported Dataframes

Creating a "helper" function to do light cleaning so we can apply it quickly on multiple dataframes.

In [32]:
def initial_clean(data):
    
    "Light cleaning on raw data by dropping unnamed column, creating identifyer column, and re-ordering columns"
    
    data = data.drop(columns=['Unnamed: 0'])  # Drop the first column
 
    return (data)

### Datasets

Pulling in all compiled datasets as csv's. We'll then compile them into dataframes later on. For now we're pulling in the csv files and setting them to variables.

In [29]:
popular1 = pd.read_csv('popular1.csv', dtype=str)
popular2 = pd.read_csv('popular2.csv', dtype=str)

In [30]:
popular1.head()

,Unnamed: 0,author,crawled,entities_locations,entities_organizations,entities_persons,external_links,highlightText,highlightTitle,language,...,thread_social_stumbledupon_shares,thread_social_vk_shares,thread_spam_score,thread_title,thread_title_full,thread_url,thread_uuid,title,url,uuid
0,0,USNews,2015-10-02T17:33:59.981+03:00,NaN,NaN,NaN,[['http://www.reddit.com/submit?url=http%3A%2F...,NaN,NaN,english,...,0,0,0.0,The Healthiest Pastas: From Quinoa to Buckwhea...,The Healthiest Pastas: From Quinoa to Buckwhea...,http://health.usnews.com/health-news/health-we...,8085f289866a814f7a443e1a31e48f8a307a040f,The Healthiest Pastas: From Quinoa to Buckwhea...,http://health.usnews.com/health-news/health-we...,8085f289866a814f7a443e1a31e48f8a307a040f
1,1,NaN,2015-10-19T09:23:00.540+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,...,0,0,0.0,Photos: Operation Santa Claus visits Savoonga,Photos: Operation Santa Claus visits Savoonga,http://www.newsdump.com/article/photos-operati...,f4ad43deab0a72726d6165b37a971c578efdd4f5,Photos: Operation Santa Claus visits Savoonga,http://www.newsdump.com/article/photos-operati...,f4ad43deab0a72726d6165b37a971c578efdd4f5
2,2,NaN,2015-10-08T17:42:28.717+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,...,0,0,0.0,"Watch: Video Shows 2,000-Year-Old Ancient Arch...","Watch: Video Shows 2,000-Year-Old Ancient Arch...",http://www.newsdump.com/article/watch-video-sh...,c98cbd870f52950ff685e772fd189bd01fc85767,"Watch: Video Shows 2,000-Year-Old Ancient Arch...",http://www.newsdump.com/article/watch-video-sh...,c98cbd870f52950ff685e772fd189bd01fc85767
3,3,NaN,2015-10-05T10:10:00.218+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,...,0,0,0.0,'Fear the Walking Dead' ends Season 1 on a gri...,'Fear the Walking Dead' ends Season 1 on a gri...,http://www.newsdump.com/article/fear-the-walki...,3481ad311613e0da31e6017f854c7ded093b398a,'Fear the Walking Dead' ends Season 1 on a gri...,http://www.newsdump.com/article/fear-the-walki...,3481ad311613e0da31e6017f854c7ded093b398a
4,4,NaN,2015-10-23T15:40:06.454+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,...,0,0,0.0,Facebook app draining your iPhone battery? Com...,Facebook app draining your iPhone battery? Com...,http://www.newsdump.com/article/facebook-app-d...,17954912c005732967b28ef81b4ebc58d3911efc,Facebook app draining your iPhone battery? Com...,http://www.newsdump.com/article/facebook-app-d...,17954912c005732967b28ef81b4ebc58d3911efc


### Dataframes

We run the helper function to every variable from the <b> dataset</b> section above and then we'll join them all together as one dataframe. 

In [33]:
popular1_df = initial_clean(popular1)
popular2_df = initial_clean(popular2)

## Compiling Dataframes

In [40]:
data = popular1_df.append(popular2_df, sort=False)

In [41]:
data.head()

,author,crawled,entities_locations,entities_organizations,entities_persons,external_links,highlightText,highlightTitle,language,locations,...,thread_social_vk_shares,thread_spam_score,thread_title,thread_title_full,thread_url,thread_uuid,title,url,uuid,thread_domain_rank
0,USNews,2015-10-02T17:33:59.981+03:00,NaN,NaN,NaN,[['http://www.reddit.com/submit?url=http%3A%2F...,NaN,NaN,english,NaN,...,0,0.0,The Healthiest Pastas: From Quinoa to Buckwhea...,The Healthiest Pastas: From Quinoa to Buckwhea...,http://health.usnews.com/health-news/health-we...,8085f289866a814f7a443e1a31e48f8a307a040f,The Healthiest Pastas: From Quinoa to Buckwhea...,http://health.usnews.com/health-news/health-we...,8085f289866a814f7a443e1a31e48f8a307a040f,NaN
1,NaN,2015-10-19T09:23:00.540+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,['Savoonga'],...,0,0.0,Photos: Operation Santa Claus visits Savoonga,Photos: Operation Santa Claus visits Savoonga,http://www.newsdump.com/article/photos-operati...,f4ad43deab0a72726d6165b37a971c578efdd4f5,Photos: Operation Santa Claus visits Savoonga,http://www.newsdump.com/article/photos-operati...,f4ad43deab0a72726d6165b37a971c578efdd4f5,NaN
2,NaN,2015-10-08T17:42:28.717+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,['Palmyra'],...,0,0.0,"Watch: Video Shows 2,000-Year-Old Ancient Arch...","Watch: Video Shows 2,000-Year-Old Ancient Arch...",http://www.newsdump.com/article/watch-video-sh...,c98cbd870f52950ff685e772fd189bd01fc85767,"Watch: Video Shows 2,000-Year-Old Ancient Arch...",http://www.newsdump.com/article/watch-video-sh...,c98cbd870f52950ff685e772fd189bd01fc85767,NaN
3,NaN,2015-10-05T10:10:00.218+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,NaN,...,0,0.0,'Fear the Walking Dead' ends Season 1 on a gri...,'Fear the Walking Dead' ends Season 1 on a gri...,http://www.newsdump.com/article/fear-the-walki...,3481ad311613e0da31e6017f854c7ded093b398a,'Fear the Walking Dead' ends Season 1 on a gri...,http://www.newsdump.com/article/fear-the-walki...,3481ad311613e0da31e6017f854c7ded093b398a,NaN
4,NaN,2015-10-23T15:40:06.454+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,NaN,...,0,0.0,Facebook app draining your iPhone battery? Com...,Facebook app draining your iPhone battery? Com...,http://www.newsdump.com/article/facebook-app-d...,17954912c005732967b28ef81b4ebc58d3911efc,Facebook app draining your iPhone battery? Com...,http://www.newsdump.com/article/facebook-app-d...,17954912c005732967b28ef81b4ebc58d3911efc,NaN


Removing columns with the exact same values because they are unneeded.

In [43]:
for col in data.columns:
    if len(data[col].unique()) == 1:
        data.drop(col,inplace = True,axis = 1)

In [44]:
data.head()

,author,crawled,entities_locations,entities_organizations,entities_persons,external_links,highlightText,highlightTitle,language,locations,...,thread_social_vk_shares,thread_spam_score,thread_title,thread_title_full,thread_url,thread_uuid,title,url,uuid,thread_domain_rank
0,USNews,2015-10-02T17:33:59.981+03:00,NaN,NaN,NaN,[['http://www.reddit.com/submit?url=http%3A%2F...,NaN,NaN,english,NaN,...,0,0.0,The Healthiest Pastas: From Quinoa to Buckwhea...,The Healthiest Pastas: From Quinoa to Buckwhea...,http://health.usnews.com/health-news/health-we...,8085f289866a814f7a443e1a31e48f8a307a040f,The Healthiest Pastas: From Quinoa to Buckwhea...,http://health.usnews.com/health-news/health-we...,8085f289866a814f7a443e1a31e48f8a307a040f,NaN
1,NaN,2015-10-19T09:23:00.540+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,['Savoonga'],...,0,0.0,Photos: Operation Santa Claus visits Savoonga,Photos: Operation Santa Claus visits Savoonga,http://www.newsdump.com/article/photos-operati...,f4ad43deab0a72726d6165b37a971c578efdd4f5,Photos: Operation Santa Claus visits Savoonga,http://www.newsdump.com/article/photos-operati...,f4ad43deab0a72726d6165b37a971c578efdd4f5,NaN
2,NaN,2015-10-08T17:42:28.717+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,['Palmyra'],...,0,0.0,"Watch: Video Shows 2,000-Year-Old Ancient Arch...","Watch: Video Shows 2,000-Year-Old Ancient Arch...",http://www.newsdump.com/article/watch-video-sh...,c98cbd870f52950ff685e772fd189bd01fc85767,"Watch: Video Shows 2,000-Year-Old Ancient Arch...",http://www.newsdump.com/article/watch-video-sh...,c98cbd870f52950ff685e772fd189bd01fc85767,NaN
3,NaN,2015-10-05T10:10:00.218+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,NaN,...,0,0.0,'Fear the Walking Dead' ends Season 1 on a gri...,'Fear the Walking Dead' ends Season 1 on a gri...,http://www.newsdump.com/article/fear-the-walki...,3481ad311613e0da31e6017f854c7ded093b398a,'Fear the Walking Dead' ends Season 1 on a gri...,http://www.newsdump.com/article/fear-the-walki...,3481ad311613e0da31e6017f854c7ded093b398a,NaN
4,NaN,2015-10-23T15:40:06.454+03:00,NaN,NaN,NaN,NaN,NaN,NaN,english,NaN,...,0,0.0,Facebook app draining your iPhone battery? Com...,Facebook app draining your iPhone battery? Com...,http://www.newsdump.com/article/facebook-app-d...,17954912c005732967b28ef81b4ebc58d3911efc,Facebook app draining your iPhone battery? Com...,http://www.newsdump.com/article/facebook-app-d...,17954912c005732967b28ef81b4ebc58d3911efc,NaN


In [46]:
data.to_csv('finaldata.csv', sep = ',')

## NLP Work

Now we have a compiled dataset, we have the choice of running processes on the full set or just subsets. Either way, the first step, however, will be to deal with null values.

In [2]:
data = pd.read_csv('finaldata.csv', dtype = str)

In [3]:
data = data.drop(columns=['Unnamed: 0'])

In [4]:
data_colnan=data.columns[data.isnull().any()]
data[data_colnan].isnull().sum()

author                                76379
entities_locations                   109139
entities_organizations               105393
entities_persons                     104007
external_links                       157547
highlightText                        170880
highlightTitle                       170880
locations                            153104
organizations                        145141
persons                              153170
text                                    361
thread_country                          657
thread_main_image                     53661
thread_participants_count              2433
thread_performance_score               2433
thread_published                       2433
thread_replies_count                   2433
thread_section_title                  27551
thread_site                            2433
thread_site_full                       2433
thread_site_section                   25529
thread_site_type                       4866
thread_social_facebook_comments 

In [5]:
data['title'] = data['title'].fillna("none")
data['text'] = data['text'].fillna("none")

### Running NLP on Text

In [6]:
count_vec = CountVectorizer()
X_train_count = count_vec.fit_transform(data.text)

In [7]:
print(count_vec.get_feature_names()[::1000])

['00', '0804', '10645598', '1099s', '12mph', '1553', '184bn', '1r4btxpxgq', '202h', '240000', '276248', '2gfor', '2njfq6i', '300158751', '300µg', '35466a848e51c1dbd36ecd', '3x_promotedstory', '45615', '4khz', '5547332', '5972', '64ymqvimj5', '704games', '790', '840x1', '900712003609', '9991pl3pbt', 'aaadhaar', 'abetted', 'acclimatise', 'adamant', 'adoringly', 'africans', 'ahm', 'akenzua', 'alegría', 'allisonpr', 'amaki', 'amongst', 'andreholland', 'annastazia', 'anukampa', 'appreciators', 'aregbesola', 'arriving', 'ashfordcastle', 'astrophes', 'atzimba', 'autója', 'ayeshashroff', 'babewatch', 'bahrun', 'bambolim', 'bargara', 'batane', 'beachland', 'behavioural', 'benefitting', 'besi', 'bhatte', 'bilirakis', 'bishphool', 'bleak', 'boafo', 'bona', 'bosintang', 'brabin', 'breira', 'brobby', 'bt14', 'bunchy', 'buttersafe', 'c94d46967414d063327750', 'callthemidwife', 'canonsburg', 'carla', 'castings', 'cchhhh', 'ceridono', 'chandaben', 'chautha', 'chiaraferragni', 'chofu', 'chuvanna', 'civi

In [12]:
lemtzer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return lemtzer.lemmatize(text, pos='v')

# Write a function to perform the pre processing steps on the entire dataset
def preprocess(text):
    result=[]
    for token in simple_preprocess(text) :
        if token not in STOPWORDS:
            result.append(lemmatize_stemming(token))
            
    return result

### WARNING! 
The code below takes some time to run. 

In [16]:
processed_docs  = []

for doc in data.text:
    processed_docs.append(preprocess(doc))

Using 'Pickle' to save progress. 

In [60]:
file_1 = "processed_docs_text"

fileObject_1 = open(file_1,'wb') 

pickle.dump(processed_docs,fileObject_1)   

fileObject_1.close()

In [49]:
dictionary = gensim.corpora.Dictionary(processed_docs)

Setting parameters for our tokens - removing words occuring more or less than criteria and retaining most frequent tokens based on a threshold. 

In [50]:
dictionary.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

Using 'Pickle' to save progress. 

In [62]:
file_2 = "dictionary_text"

fileObject_2 = open(file_2,'wb') 

pickle.dump(dictionary,fileObject_2)   

fileObject_2.close()

We're formatting for "out-of-bag."

In [51]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

Using 'Pickle' to save progress. 

In [63]:
file_3 = "bowcorpus_text"

fileObject_3 = open(file_3,'wb') 

pickle.dump(bow_corpus,fileObject_3)   

fileObject_3.close()

In [54]:
%%time
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)
lda_model.save('lda.model')

CPU times: user 4min 30s, sys: 50 s, total: 5min 20s
Wall time: 6min 23s


In [55]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.011*"pm" + 0.008*"play" + 0.007*"game" + 0.007*"vs" + 0.006*"team" + 0.006*"sport" + 0.005*"nbcsn" + 0.005*"win" + 0.005*"season" + 0.005*"csn"

Topic: 1 Word: 0.007*"police" + 0.005*"old" + 0.005*"man" + 0.004*"family" + 0.004*"photo" + 0.004*"woman" + 0.003*"post" + 0.003*"day" + 0.003*"home" + 0.003*"party"

Topic: 2 Word: 0.007*"think" + 0.006*"want" + 0.005*"women" + 0.005*"need" + 0.005*"health" + 0.004*"way" + 0.004*"life" + 0.003*"feel" + 0.003*"help" + 0.003*"get"

Topic: 3 Word: 0.061*"trump" + 0.012*"president" + 0.011*"donald" + 0.007*"campaign" + 0.005*"million" + 0.005*"care" + 0.005*"white" + 0.005*"republican" + 0.005*"plan" + 0.004*"obamacare"

Topic: 4 Word: 0.038*"ht" + 0.017*"sign" + 0.014*"email" + 0.014*"forward" + 0.012*"celtics" + 0.012*"center" + 0.011*"nhl" + 0.011*"freestyle" + 0.010*"account" + 0.010*"link"

Topic: 5 Word: 0.008*"government" + 0.005*"china" + 0.005*"minister" + 0.004*"india" + 0.004*"report" + 0.004*"country" + 0.004*"bank" 

#### Quick Visualization

In [56]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.075704  0.141282       1        1  16.709406
2     -0.089880  0.056061       2        1  15.360705
8     -0.050758  0.087768       3        1  14.877908
1     -0.064183  0.103949       4        1  11.631459
5     -0.089643  0.035981       5        1  10.517122
6     -0.080969  0.001271       6        1  10.042202
9     -0.065401 -0.120300       7        1   8.640127
3     -0.034358 -0.072456       8        1   7.208590
7      0.010051 -0.235635       9        1   3.100265
4      0.389437  0.002078      10        1   1.912218, topic_info=      Category           Freq         Term          Total  loglift  logprob
term                                                                       
1556   Default  143052.000000        trump  143052.000000  30.0000  30.0000
917    Default   84138.000000    president   84138.000000  29.0000  29.0000
13294  Default   22132.000000           ht   22132.000000  28.0000  28.0000
467    Default   76329.000000        house   76329.000000  27.0000  27.0000
769    Default   51250.000000       police   51250.000000  26.0000  26.0000
938    Default   32413.000000         sign   32413.000000  25.0000  25.0000
1168   Default   41248.000000        party   41248.000000  24.0000  24.0000
312    Default   64510.000000        right   64510.000000  23.0000  23.0000
1519   Default   26827.000000       donald   26827.000000  22.0000  22.0000
4324   Default   36044.000000           vs   36044.000000  21.0000  21.0000
1084   Default   24356.000000        email   24356.000000  20.0000  20.0000
7215   Default   21015.000000          usa   21015.000000  19.0000  19.0000
133    Default   59160.000000        leave   59160.000000  18.0000  18.0000
1100   Default   19710.000000      forward   19710.000000  17.0000  17.0000
621    Default   69591.000000           pm   69591.000000  16.0000  16.0000
340    Default   24477.000000       center   24477.000000  15.0000  15.0000
92     Default   39804.000000         game   39804.000000  14.0000  14.0000
37877  Default   26849.000000          csn   26849.000000  13.0000  13.0000
30289  Default   14529.000000           px   14529.000000  12.0000  12.0000
677    Default   32922.000000         vote   32922.000000  11.0000  11.0000
39987  Default   13983.000000          enw   13983.000000  10.0000  10.0000
471    Default   34459.000000     campaign   34459.000000   9.0000   9.0000
20672  Default   26037.000000        nbcsn   26037.000000   8.0000   8.0000
654    Default   30870.000000        sport   30870.000000   7.0000   7.0000
22526  Default   15520.000000   fullscreen   15520.000000   6.0000   6.0000
341    Default   16277.000000      clinton   16277.000000   5.0000   5.0000
3718   Default   21664.000000    committee   21664.000000   4.0000   4.0000
1221   Default   21473.000000         send   21473.000000   3.0000   3.0000
1619   Default   20418.000000          nba   20418.000000   2.0000   2.0000
137    Default   16313.000000         link   16313.000000   1.0000   1.0000
...        ...            ...          ...            ...      ...      ...
34691  Topic10    2418.497314          rem    2668.020752   3.8587  -5.4577
6735   Topic10    6172.666016          nhl    7733.954102   3.7314  -4.5207
28412  Topic10    1065.781494    villanova    1097.480347   3.9276  -6.2772
1486   Topic10    4268.108887      privacy    5913.536621   3.6308  -4.8897
1355   Topic10    1762.164673          cub    2157.789307   3.7544  -5.7743
5802   Topic10    1656.078125       pacers    1989.158203   3.7736  -5.8364
1100   Topic10    7727.254883      forward   19710.238281   3.0205  -4.2961
938    Topic10    9765.181641         sign   32413.886719   2.7571  -4.0620
1084   Topic10    7912.724609        email   24356.572266   2.8326  -4.2724
1006   Topic10    5854.998535      account   15493.658203   2.9838  -4.5736
1469   Topic10    4028.434326        clic

### NLP on Title

In [35]:
count_vec = CountVectorizer()
X_train_count = count_vec.fit_transform(data.title)

In [36]:
print(count_vec.get_feature_names()[::1000])

['00', '32811261', 'aar', 'agincourt', 'amphibious', 'aronofsky', 'ayanda', 'batumalai', 'bigs', 'boop', 'buc', 'canzieri', 'chandeliers', 'clarifying', 'conceding', 'coutinho', 'd3', 'della', 'dipali', 'dozen', 'eeo', 'epson', 'f12tdf', 'finalises', 'fraim', 'gautham', 'goodcall', 'guus', 'heathen', 'hoosiers', 'ilan', 'instructor', 'janot', 'kalie', 'kirit', 'lamu', 'lift', 'luy', 'maranello', 'medal', 'minn', 'motter', 'narcing', 'nirupa', 'odzyskać', 'outvets', 'pataca', 'piaget', 'ported', 'proposing', 'rafa', 'reds', 'rethinking', 'roselawn', 'sanjiv', 'seesaw', 'shobhaa', 'slots', 'speedy', 'stolinsky', 'surpass', 'taverne', 'tikes', 'trican', 'uncharted', 'vai', 'vonovia', 'wham', 'wwiii', 'řąřşř', 'قليل', 'กไม', 'ธนาร', 'ยหายเด', 'ากระแสหย', 'キングジムの']


In [37]:
processed_docs2  = []

for doc in data.title:
    processed_docs2.append(preprocess(doc))

Using 'Pickle' to save progress. 

In [69]:
file_4 = "processed_docs_title"

fileObject_4 = open(file_4,'wb') 

pickle.dump(processed_docs2,fileObject_4)   

fileObject_4.close()

In [39]:
dictionary2 = gensim.corpora.Dictionary(processed_docs)

Filtering out tokens by criterias - Frequency.

In [40]:
dictionary2.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

Using 'Pickle' to save progress. 

In [70]:
file_5 = "dictionary_title"

fileObject_5 = open(file_5,'wb') 

pickle.dump(dictionary2,fileObject_5)   

fileObject_5.close()

Preparing for "out-of-bag" format.

In [41]:
bow_corpus2 = [dictionary2.doc2bow(doc) for doc in processed_docs]

Using 'Pickle' to save progress. 

In [71]:
file_6 = "bowcorpus_title"

fileObject_6 = open(file_6,'wb') 

pickle.dump(bow_corpus2,fileObject_6)   

fileObject_6.close()

In [57]:
%%time
lda_model2 = gensim.models.LdaMulticore(bow_corpus2, num_topics=10, id2word=dictionary2, passes=2)
lda_model2.save('lda.model')

CPU times: user 3min 26s, sys: 42.1 s, total: 4min 8s
Wall time: 5min 25s


In [47]:
for idx, topic in lda_model2.print_topics(-1):
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.023*"house" + 0.020*"trump" + 0.010*"health" + 0.010*"care" + 0.009*"president" + 0.008*"republicans" + 0.008*"vote" + 0.008*"committee" + 0.008*"republican" + 0.007*"senate"

Topic: 1 Word: 0.007*"trump" + 0.007*"school" + 0.005*"president" + 0.005*"federal" + 0.005*"million" + 0.004*"law" + 0.004*"tax" + 0.004*"fund" + 0.004*"plan" + 0.004*"program"

Topic: 2 Word: 0.006*"women" + 0.004*"help" + 0.004*"need" + 0.004*"company" + 0.004*"use" + 0.003*"health" + 0.003*"children" + 0.003*"include" + 0.003*"study" + 0.003*"life"

Topic: 3 Word: 0.006*"play" + 0.005*"game" + 0.005*"get" + 0.004*"want" + 0.004*"think" + 0.004*"look" + 0.004*"season" + 0.004*"win" + 0.004*"team" + 0.004*"star"

Topic: 4 Word: 0.009*"government" + 0.008*"minister" + 0.007*"party" + 0.006*"india" + 0.005*"mr" + 0.004*"country" + 0.004*"china" + 0.003*"uk" + 0.003*"prime" + 0.003*"eu"

Topic: 5 Word: 0.010*"enw" + 0.009*"trump" + 0.008*"article" + 0.008*"right" + 0.008*"display" + 0.007*"com" + 

#### Quick Visualization

In [72]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model2, bow_corpus2, dictionary2)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.036149 -0.004769       1        1  17.778433
5     -0.052189  0.029922       2        1  16.137451
2     -0.134694 -0.047929       3        1  16.136486
4     -0.237592 -0.115140       4        1  12.341440
3      0.027329  0.089045       5        1  10.216951
7      0.089083  0.104355       6        1   6.723022
0     -0.011164  0.105485       7        1   6.004997
1      0.044164  0.032115       8        1   5.778714
9      0.105889  0.047759       9        1   5.690976
6      0.205323 -0.240843      10        1   3.191532, topic_info=      Category           Freq         Term          Total  loglift  logprob
term                                                                       
1556   Default  143578.000000        trump  143578.000000  30.0000  30.0000
467    Default   76509.000000        house   76509.000000  29.0000  29.0000
872    Default   40772.000000        court   40772.000000  28.0000  28.0000
917    Default   86251.000000    president   86251.000000  27.0000  27.0000
4324   Default   35031.000000           vs   35031.000000  26.0000  26.0000
621    Default   68369.000000           pm   68369.000000  25.0000  25.0000
92     Default   38691.000000         game   38691.000000  24.0000  24.0000
938    Default   34227.000000         sign   34227.000000  23.0000  23.0000
107    Default   40250.000000       health   40250.000000  22.0000  22.0000
677    Default   36193.000000         vote   36193.000000  21.0000  21.0000
37877  Default   25623.000000          csn   25623.000000  20.0000  20.0000
654    Default   29905.000000        sport   29905.000000  19.0000  19.0000
1084   Default   25669.000000        email   25669.000000  18.0000  18.0000
769    Default   50317.000000       police   50317.000000  17.0000  17.0000
1519   Default   27475.000000       donald   27475.000000  16.0000  16.0000
2752   Default   32596.000000         care   32596.000000  15.0000  15.0000
1168   Default   44846.000000        party   44846.000000  14.0000  14.0000
471    Default   36957.000000     campaign   36957.000000  13.0000  13.0000
3052   Default   17677.000000        judge   17677.000000  12.0000  12.0000
262    Default   39006.000000        white   39006.000000  11.0000  11.0000
2444   Default   32571.000000         case   32571.000000  10.0000  10.0000
1901   Default   23635.000000  information   23635.000000   9.0000   9.0000
20672  Default   25889.000000        nbcsn   25889.000000   8.0000   8.0000
293    Default   26085.000000       season   26085.000000   7.0000   7.0000
494    Default   30724.000000       attack   30724.000000   6.0000   6.0000
1619   Default   19366.000000          nba   19366.000000   5.0000   5.0000
401    Default   46182.000000         city   46182.000000   4.0000   4.0000
837    Default   29668.000000     minister   29668.000000   3.0000   3.0000
319    Default   54387.000000   government   54387.000000   2.0000   2.0000
1669   Default   22332.000000         film   22332.000000   1.0000   1.0000
...        ...            ...          ...            ...      ...      ...
2697   Topic10    4698.645508        false    7282.651855   3.0064  -5.3060
1469   Topic10    5742.907227        click    9627.352539   2.9280  -5.1054
1471   Topic10    1427.514038          cox    1730.028320   3.2525  -6.4974
1910   Topic10    5148.675293      supreme    8767.967773   2.9123  -5.2146
1487   Topic10    3917.482910      profile    6273.170410   2.9738  -5.4879
1006   Topic10    8318.907227      account   16539.187500   2.7575  -4.7348
1084   Topic10   11647.336914        email   25669.031250   2.6545  -4.3982
1909   Topic10    3902.024902       submit    6289.438477   2.9673  -5.4918
938    Topic10   14221.854492         sign   34227.179688   2.5664  -4.1985
2736   Topic10    3832.259766     sentence    6645.654785   2.8942  -5.5099
462    Topic10    6335.754883       count

# End